##### Daniela Raygadas, Caroline Harrison and Om Prakash Singh 
#### Group 11

# Credit Card Fraud Detection

### Part A: Background and Problem 

Organizations nowadays invest more in technology to prevent fraud, since fraud can be a business problem which could affect growth and reputation negatively. As a result, fraud detection is a necessary banking feature that can help identify customers who have had their banking and credit card information compromised.  Fast detection is crucial for both the customers and banks who are impacted by fraudulent transactions.  

Our objective is to apply outlier analysis to detect fraudulent credit card transactions. Missing a fraudulent transaction can be very costly for a company, that's why it is important to make sure that we don't miss any fraudulent transactions that could cause the company to lose a lot of money. Missing a fraudulent transaction is more costly to a company, than to incorrectly classifying a non-fraudulent transaction of a client as a fraudulent transaction. 

For this analysis we looked at different methods for outlier classification, and identified which of these methods was the best one at indentifying the most fraudulent transactions. 

### Part B. Data Exploration and Preparation

The dataset we found comes from Kaggle and contains information from over 284,000 real transactions, and has 30 different attributes that describe the data objects.  For privacy reasons, all but two of the data attributes have had a pca transformation applied to them, and the attribute name has been redacted.  The two attributes that were not transformed are time (seconds after first transaction in the dataset) and purchase amount(€).   Even with the data attributes pre-transformed, this dataset offers real credit card transaction information for us to perform an outlier analysis.

Link to Dataset: https://www.kaggle.com/mlg-ulb/creditcardfraud?select=creditcard.csv

In [ ]:
%matplotlib inline
import scipy.stats as stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('creditcard.csv')
df_2 = df.copy()
df.head(5)

#### First, we did some data explorationto to see how many fraudulent transactions, and non-fradulent transactions we had on the dataset. 

In [ ]:
#fraud vs. normal transactions 
counts = df.Class.value_counts()
normal = counts[0]
fraudulent = counts[1]
perc_normal = (normal/(normal+fraudulent))*100
perc_fraudulent = (fraudulent/(normal+fraudulent))*100
print('There are {} non-fraudulent transactions ({:.3f}%) and {} fraudulent transactions ({:.3f}%).'.format(normal, perc_normal, fraudulent, perc_fraudulent))

In [ ]:
sns.countplot('Class', data=df)
plt.title('Class Distributions \n No Fraud                           Fraud', fontsize=14)

In [ ]:
#plotting non-scaled data attributes
fig, ax = plt.subplots(1, 2, figsize=(18,4))

amount_val = df['Amount'].values
time_val = df['Time'].values

sns.distplot(amount_val, ax=ax[0], color='r')
ax[0].set_title('Distribution of Transaction Amount', fontsize=14)
ax[0].set_xlim([min(amount_val), max(amount_val)])

sns.distplot(time_val, ax=ax[1], color='b')
ax[1].set_title('Distribution of Transaction Time', fontsize=14)
ax[1].set_xlim([min(time_val), max(time_val)])


plt.show()

From these plots we can see that most of the transactions happened during the day time, and that the distribution for the transaction amount is positively skewed. 

#### Check for missing values in our dataset

In [ ]:
df.isna().sum()

##### Visualization to see how fraudulent and non-fraudulent transactions are scattered using the two features that weren't transformed with PCA in the original dataset.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter((df.loc[df['Class'] == 0]['Time']/(60*60)), df.loc[df['Class'] == 0]['Amount'], label="Non-fraud", alpha=0.5, linewidth=0.15,c='b')
plt.scatter((df.loc[df['Class'] == 1]['Time']/(60*60)), df.loc[df['Class'] == 1]['Amount'], label="Fraud", alpha=0.5, linewidth=0.15,c='r')

plt.title("Amount in dollars vs Transaction by hour")
plt.xlabel("Transaction time (hours)")
plt.ylabel('Amount (USD)')

plt.legend()
plt.show()

#### In this step we make sure we standarized the two feautres Time and Amount which weren't transformed with PCA in the original dataset. We know that for the other 28 features, in order to be able to perform PCA on these features they had to scaled the features first. We used the StandardScaler model to standarize these two features. 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler2 = StandardScaler()

scaled_time = scaler.fit_transform(df[['Time']])
Time_list1 = [item for sublist in scaled_time.tolist() for item in sublist]
scaled_time = pd.Series(Time_list1)

scaled_amount = scaler2.fit_transform(df[['Amount']])
Amount_list2 = [item for sublist in scaled_amount.tolist() for item in sublist]
scaled_amount = pd.Series(Amount_list2)

df = pd.concat([df, scaled_amount.rename('scaled_amount'), scaled_time.rename('scaled_time')], axis=1)
#dropping old amount and time columns
df.drop(['Amount', 'Time'], axis=1, inplace=True)

df.head(5)

In [ ]:
import math

fig, ax = plt.subplots(1, 2, figsize=(18,4))

amount_s = np.log(df['scaled_amount'].values)
time_s = df['scaled_time'].values

sns.distplot(amount_s, ax=ax[0], color='r')
ax[0].set_title('Distribution of Transaction Amount', fontsize=14)
ax[0].set_xlim([min(amount_s), max(amount_s)])

sns.distplot(time_s, ax=ax[1], color='b')
ax[1].set_title('Distribution of Transaction Time', fontsize=14)
ax[1].set_xlim([min(time_s), max(time_s)])


plt.show()

Since the distribution of the transaction amount was positively skewed and it was very hard to see if the data was normally distributed. In this step, we took the logarithm of the standarized transaction amount. As we can see, for the most part the data looks normally distributed. 

### Part C. Classification 

The methods for outlier classification are broken out into four general methods.  

1) Statistical, which assumes a normal distribution of the dataset, and assigns class labels based on that distribution. 

2) Proximity - the two types of proximity based approaches are distance based and density based.  Distance based consult the neighborhood defined by a radius of a given data object.  Density based methods classify a data object as an outlier if it has a relatively lower density compared to it's neighbors.

3) Clustering, which assumes normal data objects belong to large and dense clusters, and classifies data objects as outliers when they are far away from the center of a cluster.  

4) Classification, which is a brute-force approach to anomaly detection as many classification methods have a more difficult time with class imbalance. 

To determine which method was the best one to identify the most fraudulent transaction in our data, we used Recall as our performance metric. Our goal is to be able to identify as many fradulent transactions as we can, by reducing the number of fraudulent transactions that the model predicted as normal (False Negative).   

Our data set is highly imbalanced, so accuracy is not a reliable perfromance metric and it doesn't make sense to calculate it for our models. We could obtain a high accuracy score by just predicting everything as the majority class, which in this case would be the normal or non-fradulent transactions. Also, as mentioned in our objectives we are not as concern with incorrectly classifying a normal transaction as a fraudulent transaction.

For each method we are showing the classification report, as the confusion matrix is using the normal class (non-fraudulent transactions) to calculate the True Positive value. In the classification report we are able to see the precision, recall and f1 score for each class. For this analysis we were interested in the fraudulent class, value of 1, so at the end we are printing the precision, recall and f1 score for the only fraudulent class. 

In [ ]:
### Split the data into train and test

from sklearn.model_selection import train_test_split

y = df[['Class']]

x = df[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'scaled_amount','scaled_time']].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

### C.1 Statistical Approach 

Statistical approaches to outlier detection are used if the attributes of the dataset are assumed to be normally distributed.  If the distribution for all data attributes are normally distributed, then the data objects in the feature space will also be normally distributed.  

1. The Elliptic Envelope

Statistical approaches to outlier detection are used if the attributes of the dataset are assumed to be normally distributed. If the distribution for all data attributes are normally distributed, then the data objects in the feature space will also be normally distributed. The Elliptic Envelope method is a method fits a multivariate gaussian distribution to the data objects. The model then classifies the test data objects as either 'normal' or 'outlier'.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.covariance import EllipticEnvelope

cov = EllipticEnvelope(contamination=.1, random_state=0).fit(X_train)
class_p = cov.predict(X_test)

class_p[class_p == 1] = 0
class_p[class_p == -1] = 1

print(confusion_matrix(y_test, class_p))
print(classification_report(y_test, class_p))

prec, rec, f2, _ = precision_recall_fscore_support(y_test, class_p, beta=2, pos_label=1, average='binary')
print(f'precision is {prec: .4f}, recall is {rec: 0.4f} and F2 score is {f2: 0.4f}')

### C.2 Proximity Based Approach

#### C.2.1 Density Based 

The idea behind a density based approach to outlier detection is that normal data points occur around a dense neighborhood and anomalies are far away. 


1.  Local Outlier Factor

The local outlier factor model uses unsupervised learning for detecting outliers using a density based proximity method. The model computes the local density difference between a data point with respect to it's closest k neighbors, where k is determined by the user. The model classifies a data point as an outlier if the local density is much lower than that of it's neighbors. The model implementation here does not classify itself, but computes the negative outlier factor for each data object and labels the objects above the 95th percentile as outliers, because data instances with larger negative outlier factor scores are more likely to be outliers.


In [ ]:
### We had to make a copy of the original data frame before we transformed the Time and Amount attributes 
### based on this copy of the original data frame we had to split the data into training and testing again
y_2 = df_2[['Class']]

x_2 = df_2[['Time','V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']].to_numpy()

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(x_2, y_2, test_size=0.2, random_state=0)


In [ ]:
from sklearn.neighbors import LocalOutlierFactor

clf=LocalOutlierFactor(n_neighbors=20, algorithm='auto', leaf_size=3, metric='minkowski', p=2, metric_params=None, contamination=0.1, n_jobs=-1)
clf.fit(X_train_2)

clf.fit_predict(X_test_2)

lof = clf.negative_outlier_factor_ 

class_p=lof.copy()
class_p[lof>=np.percentile(lof,95)]=1
class_p[lof<np.percentile(lof,95)]=0

print(confusion_matrix(y_test_2, class_p))
print()
print(classification_report(y_test_2, class_p))
print()

prec, rec, f2, _ = precision_recall_fscore_support(y_test_2, class_p, beta=2, 
                                                   pos_label=1, average='binary')
print(f'precision is {prec: 0.4f}, recall is {rec: 0.4f} and F2 score is {f2: 0.4f}')

#### C.2.2 Distance Based

1. K-Nearest Neighbors

The K-Nearest Neighbor model is a lazy learning technique that bases it's classification of a data object on how it's neighbors are classified, so if the majority of the k-nearest neighbors of a data point are classified as outliers, that data point will also be classified as an outlier.  the training data objects are stored in an n-dimensional space, and when given a test data object the classifier searches for the pattern space for the k nearest training tuples to that data object.  The distance is computed using Minkowski distance.  


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_train, y_train)

class_p = neigh.predict(X_test)


print(confusion_matrix(y_test, class_p))
print()
print(classification_report(y_test, class_p))
print()
prec, rec, f2, _ = precision_recall_fscore_support(y_test, class_p, beta=2, pos_label=1, average='binary')
print(f'precision is {prec: .4f}, recall is {rec: 0.4f} and F2 score is {f2: 0.4f}')

### C.3 Clustering

1. K-means

Clustering approaches in outlier analysis assume that normal data belongs to a dense cluster, and data objects far away from those clusters are outliers. The K-Means cluster analysis tries to partition the data objects into k groups of equal size. The outliers are then identified based on the euclidean distance from the middle of the cluster, or centroid. The data objects with a distance above the 95th percentile were classified as outliers, since the data objects with a higher distance ratio were more likely to be outliers in the dataset. One of the issues with a K-Means cluster approach is that it is not guaranteed to converge at the global optimum for clustering, and often terminates at the local optimum, and results may depend on the initial selection of cluster centers.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, roc_auc_score, roc_curve, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import plot_confusion_matrix



kmeans = KMeans(n_clusters=8, random_state=42,n_jobs=-1)
kmeans.fit(X_train)

X_test_clusters = kmeans.predict(X_test)
X_test_clusters_centers = kmeans.cluster_centers_
dist = [np.linalg.norm(x-y) for x,y in zip(X_test,X_test_clusters_centers[X_test_clusters])]

class_p = np.array(dist)
class_p[dist>=np.percentile(dist,95)]=1
class_p[dist<np.percentile(dist,95)]=0

print(confusion_matrix(y_test, class_p))
print()
print(classification_report(y_test, class_p))
print()
prec, rec, f2, _ = precision_recall_fscore_support(y_test, class_p, beta=2, 
                                                   pos_label=1, average='binary')
print(f'precision is {prec: 0.4f}, recall is {rec: 0.4f} and F2 score is {f2: 0.4f}')

### C.4 Classification Based Approach

1.  Isolation Forest: 

This method focuses on identifying the anomalies instead of the normal observations by building binary (isolation) trees. In these trees, the partitions are created by randomly selecting features and the split value between the maximum and the minimum value of the selected feature. The idea is that since anomalies are less frequent that normal observations, anomalies required less partitions to be identify than the normal observations. The partitions are created until the method isolates all the anomalies points. 


In [ ]:
from sklearn.ensemble import IsolationForest

model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.01, max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0,behaviour="new")
model.fit(X_train)

class_p = model.predict(X_test)

class_p[class_p == 1] = 0
class_p[class_p == -1] = 1

print(confusion_matrix(y_test, class_p))
print()
print(classification_report(y_test, class_p))
print()
prec, rec, f2, _ = precision_recall_fscore_support(y_test, class_p, beta=2, 
                                                   pos_label=1, average='binary')
print(f'precision is {prec: 0.4f}, recall is {rec: 0.4f} and F2 score is {f2: 0.4f}')

2. One-Class Support Vector Machine: 

This method is an unsupervised learning algorithm that is trained only on one type of class. This method is very helpful when the data is highly imbalanced like ours. In this One-Class Support Vector Machine we train the model on the majority class which in this case is the non-fraudulent class. 

In [ ]:
np.random.seed(78)
train_index = np.random.rand(len(df)) < 0.8
# use the above list to take the training (true values)
train = df[train_index]
# the opposite (rest) is for testing
test = df[~train_index]
# Check how many samples we have for training and testing
print(len(train))
print(len(test))

In [ ]:
# Get the proportion of outliers on the training set. We use this value for the nu parameter in the model,
#as the nu parameter should be the proportion of outliers you expect to observe
train_normal = train[train['Class']==0] 
train_fraud = train[train['Class']==1] 
outlier_prop = len(train_fraud) / len(train_normal) 
print(outlier_prop)

In [ ]:
from sklearn.svm import OneClassSVM
ocsvm =OneClassSVM(kernel='rbf', nu=outlier_prop,gamma=0.007)
ocsvm.fit(train_normal[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','scaled_amount','scaled_time']])

In [ ]:
test_x = test[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','scaled_amount','scaled_time']]
test_y = test['Class']
Class_pre = ocsvm.predict(test_x)

In [ ]:
## Check to see how many transactions were predicted as non-fraud and as fraud
print(len(Class_pre[Class_pre == -1]))
print(len(Class_pre[Class_pre == 1]))

In [ ]:
## Check to make sure when we changed the values to match the labels of the class in the test data, that we have the 
## same number of transactions

Class_pre[Class_pre == 1] = 0
Class_pre[Class_pre == -1] = 1
print(len(Class_pre[Class_pre == 1]))
print(len(Class_pre[Class_pre == 0]))

In [ ]:

print(confusion_matrix(test_y,Class_pre))
print()

print(classification_report(test_y, Class_pre))

prec, rec, f2, _ = precision_recall_fscore_support(test_y, Class_pre, beta=2, 
                                                   pos_label=1, average='binary')
print(f'precision is {prec: 0.4f}, recall is {rec: 0.4f} and F2 score is {f2: 0.4f}')


### Part D. Model Evaluation

|  Model                 |  Recall  |
|------------------------|-------------|
| The Elliptic Envelope |    0.8911       |    
| Local Outlier Factor|   0.0297         |  
| K-Nearest Neighbors|    0.8416        |  
| K-means|    0.9109        |  
| Isolation Forest|   0.5743         |  
| One-Class Support Vector Machine|    0.5882        |  

Elliptic Envelope: This method performed relatively well for our analysis. This may be because our data has already been PCA transformed so each of the data attributes more easily clustered to create a normal distribution in the feature space. The Elliptic Envelope method also performs well when there is high number of data objects compared to the number of features in the dataset.

Local-Outlier Factor: This model ended up not performing well for our dataset, with an accuracy of only 94%, low in comparison to other models tested, and a recall scores of 2.97%, very low compared to other models tested. We ended up using the non-transformed data to train and test this model, as our first test with the standard scaled data performed even worse than the non-transformed data. Possible reasons why this model performed so poorly with our dataset is that there is an assumption that the density of an outlier is significantly different than the density of its neighbors, and the differences in density may not have been significant enough to make outlier detection very clear with this dataset. Possibly because the attributes of the original dataset were already pca normalized.

K-Nearest Neighbors: This model performed fairly well in comparison to the rest of the classification techniques in our analysis.  The best performance came when we set the n_neighbors parameter to 1, possibly because there are too few outliers being detected that having your closest neighbor be classified as a normal data point be a strong enough threshold to support classifying that data point as normal as well.  

K-means: This model performed the best in our analysis, this may be because the PCA transformation of the dataset prior to our analysis already extracted some kind of cluster structure within the data, and the k-means analysis was then more easily able to create clusters of a fairly regular shape. Often times, the reason k-means cluster models underperform for outlier detection because clusters of irregular shape can skew the classification.

Isolation Forest: This method did not perform as well as other methods that we tried. This could have happened if the normal and anomalous observations were close to each other meaning that they could have been located in the same dense area. This will lead to the model wrongly classifying anamalous instances as normal.

One-Class SVM: This method did not perform as well as other methods we tried. Even when we tried to train the model using a mixture of non-fradulent transactions and fradulent transactions, the performance did not improve. Even when we decreased the gamma parameter, the model did not seemed to improve. A decrease in training error is to be expected by decreasing nu, since more and more training data points fall on the correct side of the hypersphere. But assuming a too small outlier ratio can easily result in many negatives also falling on this side of the hypersphere, causing a higher test error. 


### PART E. CONCLUSION



The model that performed the best in our analysis was K-means with a recall score of 91.09. This model was successful at identifying the most fraudulent transactions in our dataset. As mentioned above, this might had happened since the PCA transformation of the dataset already extracted some kind of cluster structure within the data. This allowed the model to be able to create clusters of a fairly regular shape. 

Missing a fraudulent transaction can be very costly for a company, so by making sure that the recall is as high as possible we are ensuring that this model minimized the number of fraudulent transactions predicted to be non-fraudulent transactions. 